# Exercise 16


In [1]:
import pandas as pd
train_df = pd.read_csv('14_mashable_train_df.csv', index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2013/09/16/sprints-upgrade...,479,11,1045,0.407946,1,0.617021,29,3,1,...,0.033333,1.0,-0.305139,-0.700000,-0.125000,0.0,0.0,0.5,0.0,1
1,http://mashable.com/2014/12/06/eric-garner-pro...,31,14,0,0.000000,0,0.000000,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,-0.5,0.5,0.5,0
2,http://mashable.com/2013/06/06/human-scale-inv...,581,6,604,0.470000,1,0.675516,9,0,1,...,0.100000,1.0,-0.215873,-0.666667,-0.071429,0.1,0.0,0.4,0.0,0
3,http://mashable.com/2014/11/24/beyonce-spotify...,44,16,708,0.474286,1,0.675325,5,2,2,...,0.062500,0.5,-0.376852,-0.875000,-0.100000,0.4,-0.2,0.1,0.2,0
4,http://mashable.com/2013/03/03/superheroic-let...,676,7,30,1.000000,1,1.000000,2,1,0,...,0.250000,0.7,0.000000,0.000000,0.000000,0.0,0.0,0.5,0.0,1


In [2]:
train_df.shape

(23786, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular'].values

# Exercice 16.1 (2 points)

Estimate a Logistic Regression, GaussianNB, K-nearest neighbors and a Decision Tree **Classifiers**

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=2)

Comment about the results

Combine the classifiers and comment

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

models = {'lr': LogisticRegression(),
          'dt': DecisionTreeClassifier(),
          'nb': GaussianNB(),
          'nn': KNeighborsClassifier()}

In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

for model in models.keys():
    models[model].fit(X_train, y_train)
    

In [6]:
# predict test for each model
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [7]:
# Evaluate each model
import numpy as np
from sklearn.metrics import mean_squared_error

for model in models.keys():
    print(model,np.sqrt(mean_squared_error(y_pred[model], y_test)))

lr 0.358189870299
nn 0.371554571673
nb 0.393959775474
dt 0.461936034478


In [8]:
y_pred.head()

,lr,nn,nb,dt
4754,0,0,0,0
10360,0,0,0,0
13048,0,0,0,0
1912,0,0,1,0
3323,0,0,0,0


In [9]:
from sklearn import metrics
for model in models.keys():
    print(model, 'precision',metrics.precision_score(y_pred[model],y_test))
    print(model, 'recall', metrics.recall_score(y_pred[model],y_test))
    print(model, 'f1.score',metrics.f1_score(y_pred[model],y_test))
    print(model, 'fbeta',metrics.fbeta_score(y_pred[model],y_test,beta=4))

lr precision 0.00264900662252
lr recall 0.166666666667
lr f1.score 0.00521512385919
lr fbeta 0.0359028511088
nn precision 0.0331125827815
nn recall 0.215517241379
nn f1.score 0.0574052812859
nn fbeta 0.162772883953
nb precision 0.104635761589
nb recall 0.242331288344
nb f1.score 0.146160962072
nb fbeta 0.224920448836
dt precision 0.225165562914
dt recall 0.19906323185
dt f1.score 0.211311373524
dt fbeta 0.20042998821


La precisión de los arboles de decision es significativamente mejor que los demás metodos. El mejor recall que es la tasa de verdaderos positivos es de naive bayes.
Los arboles de decisión también tienen el mejor f1.score y el mejor fbeta es de naive bayes.
Con esto podemos ver que a regresión logistica no tiene un buen ajuste en este caso al igual que KNN.
Naive Bayes genera buenos resultados pero definitivamente los arboles de decisión son los que mejor se ajustan a los datos.

# Exercice 16.2

Apply random-undersampling with a target percentage of 0.5

how does the results change

In [10]:
def UnderSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y == 1)
    filter_ = filter_.astype(bool)
    
    return X[filter_], y[filter_]

In [11]:
X_u, y_u = UnderSampling(X_train, y_train,0.5,666)

for model in models.keys():
    models[model].fit(X_u, y_u)
    
# predict test for each model
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)
    

from sklearn import metrics
for model in models.keys():
    print(model, 'fbeta',metrics.fbeta_score(y_pred[model],y_test,beta=2))

lr fbeta 0.244509358433
nn fbeta 0.193039443155
nb fbeta 0.216116085211
dt fbeta 0.180312311276


Con este metodo la regresión logistica obtiene resultados muy buenos comparados con los que tenia ya que pasa de tener un fbeta de 0.035 a uno de 0.244 lo que es un cambio significativo, además se vuelve el de mejor puntaje (en fbeta).

# Exercice 16.3 (3 points)

For each model estimate a BaggingClassifier of 100 models using the under sampled datasets

In [12]:
from sklearn.tree import DecisionTreeClassifier

treeclas = DecisionTreeClassifier(max_depth=None, random_state=123)


from sklearn.ensemble import BaggingClassifier
bagreg = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

In [13]:
# fit and predict
bagreg.fit(X_u, y_u)
y_pred = bagreg.predict(X_test)
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [14]:
from sklearn import metrics
print('fbeta',metrics.fbeta_score(y_pred,y_test,beta=2))

fbeta 0.251202041017


# Exercice 16.4 (2 points)

Using the under-sampled dataset

Evaluate a RandomForestClassifier and compare the results

change n_estimators=100, what happened

In [15]:
from sklearn.ensemble import RandomForestClassifier
rfreg = RandomForestClassifier(n_estimators=100, random_state=1, n_jobs=-1)

In [16]:
rfreg.fit(X_u, y_u)
y_pred = rfreg.predict(X_test)
y_pred

array([1, 0, 1, ..., 0, 1, 1])

In [17]:
print('fbeta',metrics.fbeta_score(y_pred,y_test,beta=2))

fbeta 0.246974318607


In [18]:
pd.DataFrame({'feature':X.columns, 'importance':rfreg.feature_importances_}).sort_values('importance')

,feature,importance
4,n_non_stop_words,0.000173
12,data_channel_is_lifestyle,0.001470
35,weekday_is_saturday,0.002036
36,weekday_is_sunday,0.002550
15,data_channel_is_socmed,0.002713
16,data_channel_is_tech,0.002837
14,data_channel_is_bus,0.002911
13,data_channel_is_entertainment,0.002940
33,weekday_is_thursday,0.003072
37,is_weekend,0.003098


In [19]:
x_u_=rfreg.transform(X_u, threshold='mean')

/Users/FelipeGarcia/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


In [20]:
x_u_.shape

(4683, 29)

In [21]:
X_test_ = rfreg.transform(X_test, threshold='mean')

/Users/FelipeGarcia/anaconda/lib/python3.5/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
